# 🎬 SVD 空间氛围视频生成

将室内效果图转换为动态视频展示

**使用方法：**
1. 点击 `运行时` → `更改运行时类型` → 选择 `T4 GPU`
2. 按顺序运行每个单元格
3. 上传你的室内效果图
4. 等待生成完成，下载视频

In [ ]:
#@title 1️⃣ 安装依赖（首次运行需要2-3分钟）
!pip install -q diffusers transformers accelerate
print("✅ 依赖安装完成")

In [ ]:
#@title 2️⃣ 检查 GPU
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU 可用: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    print("❌ 未检测到 GPU，请点击 运行时 → 更改运行时类型 → 选择 T4 GPU")

In [ ]:
#@title 3️⃣ 加载 SVD 模型（首次运行需要5-10分钟下载）
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import export_to_video
import torch

print("正在加载模型...")
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt",
    torch_dtype=torch.float16,
    variant="fp16"
)
pipe.to("cuda")
pipe.enable_model_cpu_offload()
print("✅ 模型加载完成")

In [ ]:
#@title 4️⃣ 上传室内效果图
from google.colab import files
from PIL import Image
import io

print("请上传一张室内效果图...")
uploaded = files.upload()

# 获取上传的文件
filename = list(uploaded.keys())[0]
image = Image.open(io.BytesIO(uploaded[filename])).convert("RGB")

# 调整尺寸到 1024x576
image = image.resize((1024, 576))
print(f"✅ 图片已加载: {image.size}")
display(image)

In [ ]:
#@title 5️⃣ 生成空间氛围视频 🎬
#@markdown ### 参数设置
num_frames = 25 #@param {type:"slider", min:14, max:25, step:1}
motion_level = 100 #@param {type:"slider", min:50, max:200, step:10}
seed = 42 #@param {type:"integer"}

print(f"正在生成 {num_frames} 帧视频，运动幅度: {motion_level}...")
print("预计需要 2-5 分钟，请耐心等待...")

generator = torch.Generator(device="cuda").manual_seed(seed)

frames = pipe(
    image,
    num_frames=num_frames,
    motion_bucket_id=motion_level,
    noise_aug_strength=0.02,
    decode_chunk_size=8,
    generator=generator,
).frames[0]

# 保存视频
output_path = "atmosphere_video.mp4"
export_to_video(frames, output_path, fps=7)
print(f"✅ 视频生成完成: {output_path}")

In [ ]:
#@title 6️⃣ 预览和下载视频
from IPython.display import HTML
from base64 import b64encode

# 预览视频
with open(output_path, "rb") as f:
    video_data = f.read()
    video_b64 = b64encode(video_data).decode()

display(HTML(f'''
<video width="640" controls>
  <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
</video>
'''))

# 下载视频
files.download(output_path)
print("✅ 视频已开始下载")